# Correlates of Protection (CoP) Statistical Analysis Plan

**Author:** Statistical Analysis Team  
**Date:** July 2025  
**Version:** 1.0

---

## Table of Contents
1. [Library Imports](#1-library-imports)
2. [Study Overview and Objectives](#2-study-overview-and-objectives)
3. [Study Design and Endpoints](#3-study-design-and-endpoints)
4. [Statistical Methods](#4-statistical-methods)
5. [Data Simulation](#5-data-simulation)
6. [Statistical Analysis in R](#6-statistical-analysis-in-r)
7. [Results Interpretation](#7-results-interpretation)
8. [Best Practices and Recommendations](#8-best-practices-and-recommendations)

---

## 1. Library Imports

We'll load the necessary R packages for our correlates of protection analysis.

In [ ]:
# Load required libraries
library(tidyverse)    # Data manipulation and visualization
library(survival)     # Survival analysis
library(survminer)    # Survival plot enhancements
library(pROC)         # ROC curve analysis
library(gridExtra)    # Multiple plot arrangements
library(corrplot)     # Correlation plots
library(ggplot2)      # Advanced plotting
library(dplyr)        # Data manipulation
library(broom)        # Model summaries
library(knitr)        # Table formatting
library(kableExtra)   # Enhanced table formatting
library(boot)         # Bootstrap methods
library(MASS)         # Statistical functions
library(car)          # Regression diagnostics

# Set random seed for reproducibility
set.seed(42)

# Print session info
cat("R Session Info:\n")
print(sessionInfo())

## 2. Study Overview and Objectives

### 2.1 Background

Correlates of Protection (CoP) studies are essential in vaccine development and infectious disease research. These studies aim to identify immunological markers that predict protection against infection or disease. Understanding CoP helps in:

- **Vaccine development**: Identifying biomarkers that predict vaccine efficacy
- **Regulatory approval**: Providing evidence for vaccine licensing
- **Public health decisions**: Informing vaccination strategies
- **Clinical trial design**: Optimizing future studies

### 2.2 Study Objectives

#### Primary Objective
To identify immunological correlates of protection against [specific pathogen/disease] and establish threshold levels associated with protection.

#### Secondary Objectives
1. Evaluate the relationship between antibody levels and protection
2. Assess the durability of immune responses
3. Identify factors that influence immune response variability
4. Develop predictive models for protection

### 2.3 Study Population

- **Target Population**: Healthy adults aged 18-65 years
- **Sample Size**: 500 participants
- **Follow-up Period**: 12 months
- **Study Design**: Prospective cohort study

## 3. Study Design and Endpoints

### 3.1 Study Design

This is a prospective cohort study designed to identify correlates of protection. Participants will be followed for 12 months after vaccination to assess:

1. **Immunological markers** (antibody levels, T-cell responses)
2. **Clinical outcomes** (infection, disease severity)
3. **Demographic and clinical covariates**

### 3.2 Primary Endpoint

**Laboratory-confirmed infection** within 12 months of vaccination
- Binary outcome: Yes/No
- Confirmed by RT-PCR or antigen testing

### 3.3 Secondary Endpoints

1. **Time to infection** (survival analysis)
2. **Disease severity** (ordinal scale: asymptomatic, mild, moderate, severe)
3. **Antibody persistence** (continuous measures over time)
4. **Breakthrough infection rate** by antibody quartiles

### 3.4 Immunological Assessments

1. **Neutralizing antibodies** (primary biomarker)
2. **Binding antibodies** (IgG, IgA, IgM)
3. **T-cell responses** (CD4+, CD8+)
4. **Cytokine profiles**

### 3.5 Timeline

- **Baseline** (Day 0): Pre-vaccination
- **Day 28**: Post-vaccination (peak response)
- **Month 3, 6, 9, 12**: Follow-up visits
- **Unscheduled visits**: Symptomatic illness

## 4. Statistical Methods

### 4.1 General Principles

- **Significance Level**: α = 0.05 (two-sided)
- **Missing Data**: Multiple imputation for missing immunological data
- **Multiple Testing**: Benjamini-Hochberg correction for multiple comparisons
- **Analysis Populations**: 
  - Intent-to-treat (ITT)
  - Per-protocol (PP)
  - Immunogenicity evaluable population

### 4.2 Correlates of Protection Analysis

#### 4.2.1 Logistic Regression
Primary method for identifying correlates of protection:

$$\text{logit}(P(\text{protection})) = \beta_0 + \beta_1 \log(\text{antibody}) + \beta_2 \text{age} + \beta_3 \text{sex} + \ldots$$

#### 4.2.2 Survival Analysis
Cox proportional hazards model for time-to-infection:

$$h(t|X) = h_0(t) \exp(\beta_1 \log(\text{antibody}) + \beta_2 X_2 + \ldots)$$

#### 4.2.3 ROC Analysis
Receiver Operating Characteristic curves to:
- Assess discriminatory ability
- Determine optimal threshold levels
- Calculate sensitivity and specificity

#### 4.2.4 Threshold Analysis
Methods to establish protection thresholds:
- **Quantile regression**: 50%, 80%, 90% protection levels
- **Prentice criteria**: Statistical framework for CoP validation
- **Bootstrap confidence intervals**: Uncertainty quantification

### 4.3 Sample Size Calculation

For logistic regression with 80% power, α = 0.05:
- **Expected attack rate**: 20%
- **Detectable OR**: 0.5 per log10 increase in antibody
- **Required sample size**: 500 participants

### 4.4 Statistical Software

Primary analysis will be conducted in R using:
- Base R statistical functions
- Specialized packages for survival analysis, ROC curves
- Custom functions for CoP-specific analyses

## 5. Data Simulation

To demonstrate the statistical methods, we'll simulate a realistic dataset that represents a correlates of protection study.

In [ ]:
# Simulate correlates of protection data
simulate_cop_data <- function(n = 500, seed = 42) {
  set.seed(seed)
  
  # Baseline characteristics
  age <- round(rnorm(n, mean = 45, sd = 15))
  age[age < 18] <- 18
  age[age > 65] <- 65
  
  sex <- sample(c("Male", "Female"), n, replace = TRUE, prob = c(0.48, 0.52))
  
  # Comorbidities
  diabetes <- rbinom(n, 1, 0.12)
  hypertension <- rbinom(n, 1, 0.25)
  bmi <- round(rnorm(n, mean = 26, sd = 4), 1)
  
  # Vaccination history
  prior_vaccination <- rbinom(n, 1, 0.15)
  
  # Immunological responses (post-vaccination)
  # Log-normal distribution for antibody levels
  log_antibody_mean <- 2.5 + 0.02 * (age - 45) + 0.3 * (sex == "Female") - 0.2 * diabetes
  log_antibody <- rnorm(n, mean = log_antibody_mean, sd = 0.8)
  antibody_level <- exp(log_antibody)
  
  # T-cell responses (correlated with antibody)
  t_cell_response <- exp(0.5 * log_antibody + rnorm(n, 0, 0.3))
  
  # Infection outcome (based on antibody levels and other factors)
  # Higher antibody levels = lower infection risk
  logit_infection <- -1.5 - 0.8 * log_antibody + 0.01 * age + 0.3 * diabetes + 0.2 * hypertension
  infection_prob <- 1 / (1 + exp(-logit_infection))
  infected <- rbinom(n, 1, infection_prob)
  
  # Time to infection (for survival analysis)
  # Those who don't get infected are censored at 365 days
  lambda <- exp(-2 - 0.5 * log_antibody)  # Hazard rate
  time_to_infection <- rexp(n, lambda)
  time_to_infection[infected == 0] <- NA
  
  # Create follow-up time (censoring time)
  follow_up_time <- ifelse(infected == 1 & !is.na(time_to_infection), 
                          pmin(time_to_infection, 365), 365)
  
  # Disease severity (among infected)
  # Lower antibody levels = higher severity
  severity_prob <- ifelse(infected == 1, 
                         pmax(0.1, 0.7 - 0.2 * log_antibody), 0)
  severity <- ifelse(infected == 1, 
                    sample(c("Asymptomatic", "Mild", "Moderate", "Severe"), 
                           n, replace = TRUE, 
                           prob = c(0.3, 0.5, 0.15, 0.05)), NA)
  
  # Create the dataset
  data <- data.frame(
    subject_id = 1:n,
    age = age,
    sex = sex,
    bmi = bmi,
    diabetes = diabetes,
    hypertension = hypertension,
    prior_vaccination = prior_vaccination,
    antibody_level = round(antibody_level, 2),
    log_antibody = round(log_antibody, 3),
    t_cell_response = round(t_cell_response, 2),
    infected = infected,
    follow_up_time = round(follow_up_time, 0),
    time_to_infection = round(time_to_infection, 0),
    severity = severity,
    stringsAsFactors = FALSE
  )
  
  return(data)
}

# Generate the dataset
cop_data <- simulate_cop_data(n = 500)

# Display first few rows
head(cop_data, 10) %>%
  kable(caption = "First 10 rows of simulated CoP dataset") %>%
  kable_styling(bootstrap_options = c("striped", "hover"))

# Summary statistics
cat("\n=== DATASET SUMMARY ===\n")
cat("Total participants:", nrow(cop_data), "\n")
cat("Number infected:", sum(cop_data$infected), "\n")
cat("Attack rate:", round(mean(cop_data$infected) * 100, 1), "%\n")
cat("Mean antibody level:", round(mean(cop_data$antibody_level), 2), "\n")
cat("Median follow-up time:", median(cop_data$follow_up_time), "days\n")

### 5.1 Data Exploration and Visualization

In [ ]:
# Descriptive statistics by infection status
descriptive_stats <- cop_data %>%
  group_by(infected) %>%
  summarise(
    n = n(),
    mean_age = round(mean(age), 1),
    mean_antibody = round(mean(antibody_level), 2),
    median_antibody = round(median(antibody_level), 2),
    sd_antibody = round(sd(antibody_level), 2),
    pct_female = round(mean(sex == "Female") * 100, 1),
    pct_diabetes = round(mean(diabetes) * 100, 1)
  )

descriptive_stats %>%
  kable(caption = "Descriptive Statistics by Infection Status") %>%
  kable_styling(bootstrap_options = c("striped", "hover"))

# Visualization: Antibody levels by infection status
p1 <- ggplot(cop_data, aes(x = factor(infected), y = antibody_level, fill = factor(infected))) +
  geom_boxplot(alpha = 0.7) +
  geom_jitter(width = 0.2, alpha = 0.5) +
  scale_y_log10() +
  labs(title = "Antibody Levels by Infection Status",
       x = "Infected (0 = No, 1 = Yes)",
       y = "Antibody Level (log scale)",
       fill = "Infected") +
  theme_minimal() +
  theme(legend.position = "none")

# Correlation matrix
cor_vars <- cop_data %>%
  select(age, antibody_level, t_cell_response, infected) %>%
  cor(use = "complete.obs")

# Create correlation plot
p2 <- corrplot(cor_vars, method = "color", type = "upper", 
               order = "hclust", tl.cex = 0.8, tl.col = "black")

# Display plots
print(p1)

# Antibody distribution
p3 <- ggplot(cop_data, aes(x = antibody_level)) +
  geom_histogram(bins = 30, alpha = 0.7, fill = "steelblue") +
  scale_x_log10() +
  labs(title = "Distribution of Antibody Levels",
       x = "Antibody Level (log scale)",
       y = "Frequency") +
  theme_minimal()

print(p3)

## 6. Statistical Analysis in R

### 6.1 Primary Analysis: Logistic Regression for Correlates of Protection

In [ ]:
# Primary logistic regression model
# Outcome: infection (0 = protected, 1 = infected)
# We model infection risk, so lower antibody = higher risk

# Univariate analysis
model_univariate <- glm(infected ~ log_antibody, 
                       data = cop_data, 
                       family = binomial)

# Multivariate analysis
model_multivariate <- glm(infected ~ log_antibody + age + sex + diabetes + hypertension,
                         data = cop_data,
                         family = binomial)

# Model summaries
cat("=== UNIVARIATE LOGISTIC REGRESSION ===\n")
summary(model_univariate)

cat("\n=== MULTIVARIATE LOGISTIC REGRESSION ===\n")
summary(model_multivariate)

# Extract odds ratios and confidence intervals
univariate_or <- exp(cbind(coef(model_univariate), confint(model_univariate)))
multivariate_or <- exp(cbind(coef(model_multivariate), confint(model_multivariate)))

cat("\n=== ODDS RATIOS (95% CI) ===\n")
cat("Univariate - Log Antibody:\n")
print(round(univariate_or, 3))

cat("\nMultivariate Model:\n")
print(round(multivariate_or, 3))

# Calculate vaccine efficacy by antibody quartiles
cop_data$antibody_quartile <- cut(cop_data$antibody_level, 
                                 breaks = quantile(cop_data$antibody_level, 
                                                  c(0, 0.25, 0.5, 0.75, 1)), 
                                 labels = c("Q1", "Q2", "Q3", "Q4"),
                                 include.lowest = TRUE)

infection_by_quartile <- cop_data %>%
  group_by(antibody_quartile) %>%
  summarise(
    n = n(),
    infections = sum(infected),
    attack_rate = round(mean(infected) * 100, 1),
    .groups = 'drop'
  )

infection_by_quartile %>%
  kable(caption = "Infection Rate by Antibody Quartile") %>%
  kable_styling(bootstrap_options = c("striped", "hover"))

### 6.2 ROC Analysis and Threshold Determination

In [ ]:
# ROC analysis for antibody levels
# Note: we use 1-infected as outcome since we want to predict protection
roc_analysis <- roc(response = 1 - cop_data$infected, 
                   predictor = cop_data$antibody_level)

# Calculate AUC and confidence interval
auc_value <- auc(roc_analysis)
auc_ci <- ci.auc(roc_analysis)

cat("=== ROC ANALYSIS RESULTS ===\n")
cat("AUC:", round(auc_value, 3), "\n")
cat("95% CI:", round(auc_ci[1], 3), "-", round(auc_ci[3], 3), "\n")

# Find optimal threshold using Youden's J statistic
optimal_threshold <- coords(roc_analysis, "best", ret = "threshold")
optimal_coords <- coords(roc_analysis, "best", ret = c("threshold", "sensitivity", "specificity"))

cat("\n=== OPTIMAL THRESHOLD ===\n")
cat("Threshold:", round(optimal_threshold, 2), "\n")
cat("Sensitivity:", round(optimal_coords$sensitivity, 3), "\n")
cat("Specificity:", round(optimal_coords$specificity, 3), "\n")

# Plot ROC curve
roc_plot <- ggroc(roc_analysis) +
  geom_abline(intercept = 1, slope = 1, linetype = "dashed", color = "red") +
  labs(title = paste0("ROC Curve for Antibody Levels\nAUC = ", round(auc_value, 3)),
       x = "Specificity",
       y = "Sensitivity") +
  theme_minimal()

print(roc_plot)

# Calculate protection levels at different thresholds
thresholds <- c(10, 20, 50, 100, 200, 500)
protection_analysis <- data.frame(
  threshold = thresholds,
  n_above = sapply(thresholds, function(x) sum(cop_data$antibody_level >= x)),
  pct_above = sapply(thresholds, function(x) round(mean(cop_data$antibody_level >= x) * 100, 1)),
  attack_rate_above = sapply(thresholds, function(x) {
    subset_data <- cop_data[cop_data$antibody_level >= x, ]
    if(nrow(subset_data) > 0) {
      round(mean(subset_data$infected) * 100, 1)
    } else {
      NA
    }
  }),
  attack_rate_below = sapply(thresholds, function(x) {
    subset_data <- cop_data[cop_data$antibody_level < x, ]
    if(nrow(subset_data) > 0) {
      round(mean(subset_data$infected) * 100, 1)
    } else {
      NA
    }
  })
)

protection_analysis %>%
  kable(caption = "Protection Analysis by Antibody Threshold") %>%
  kable_styling(bootstrap_options = c("striped", "hover"))

### 6.3 Survival Analysis

In [ ]:
# Prepare survival data
# Create event indicator (1 = infected, 0 = censored)
cop_data$event <- cop_data$infected
cop_data$time <- cop_data$follow_up_time

# Create survival object
surv_object <- Surv(time = cop_data$time, event = cop_data$event)

# Kaplan-Meier survival curves by antibody quartiles
km_fit <- survfit(surv_object ~ antibody_quartile, data = cop_data)

# Plot survival curves
km_plot <- ggsurvplot(
  km_fit,
  data = cop_data,
  pval = TRUE,
  conf.int = TRUE,
  risk.table = TRUE,
  risk.table.col = "strata",
  linetype = "strata",
  surv.median.line = "hv",
  ggtheme = theme_minimal(),
  palette = c("#E7B800", "#2E9FDF", "#00AFBB", "#FC4E07"),
  title = "Survival Curves by Antibody Quartile",
  xlab = "Time (days)",
  ylab = "Survival Probability (Infection-free)"
)

print(km_plot)

# Cox proportional hazards model
cox_model <- coxph(surv_object ~ log_antibody + age + sex + diabetes + hypertension,
                   data = cop_data)

cat("\n=== COX PROPORTIONAL HAZARDS MODEL ===\n")
summary(cox_model)

# Test proportional hazards assumption
cox_zph <- cox.zph(cox_model)
cat("\n=== PROPORTIONAL HAZARDS TEST ===\n")
print(cox_zph)

# Calculate hazard ratios
cox_hr <- exp(cbind(coef(cox_model), confint(cox_model)))
cat("\n=== HAZARD RATIOS (95% CI) ===\n")
print(round(cox_hr, 3))

### 6.4 Dose-Response Analysis

In [ ]:
# Dose-response curve
# Create prediction data
antibody_range <- seq(min(cop_data$antibody_level), max(cop_data$antibody_level), length.out = 100)
pred_data <- data.frame(
  log_antibody = log(antibody_range),
  age = mean(cop_data$age),
  sex = "Female",
  diabetes = 0,
  hypertension = 0
)

# Predict infection probability
pred_prob <- predict(model_multivariate, newdata = pred_data, type = "response")
pred_protection <- 1 - pred_prob

# Create dose-response plot
dose_response_data <- data.frame(
  antibody_level = antibody_range,
  protection_prob = pred_protection
)

dose_response_plot <- ggplot(dose_response_data, aes(x = antibody_level, y = protection_prob)) +
  geom_line(size = 1.2, color = "blue") +
  geom_hline(yintercept = c(0.5, 0.8, 0.9), linetype = "dashed", alpha = 0.6) +
  scale_x_log10() +
  labs(
    title = "Dose-Response Curve: Antibody Level vs Protection Probability",
    x = "Antibody Level (log scale)",
    y = "Protection Probability",
    caption = "Dashed lines at 50%, 80%, and 90% protection levels"
  ) +
  theme_minimal()

print(dose_response_plot)

# Calculate antibody levels for specific protection levels
protection_levels <- c(0.5, 0.8, 0.9)
antibody_thresholds <- sapply(protection_levels, function(target) {
  idx <- which.min(abs(pred_protection - target))
  antibody_range[idx]
})

threshold_table <- data.frame(
  protection_level = paste0(protection_levels * 100, "%"),
  antibody_threshold = round(antibody_thresholds, 2)
)

threshold_table %>%
  kable(caption = "Antibody Thresholds for Different Protection Levels") %>%
  kable_styling(bootstrap_options = c("striped", "hover"))

### 6.5 Bootstrap Analysis for Confidence Intervals

In [ ]:
# Bootstrap analysis for threshold confidence intervals
bootstrap_threshold <- function(data, indices, protection_level = 0.8) {
  # Sample with replacement
  boot_data <- data[indices, ]
  
  # Fit model
  tryCatch({
    model <- glm(infected ~ log_antibody + age + sex + diabetes + hypertension,
                data = boot_data, family = binomial)
    
    # Create prediction data
    antibody_range <- seq(min(boot_data$antibody_level), 
                         max(boot_data$antibody_level), length.out = 100)
    pred_data <- data.frame(
      log_antibody = log(antibody_range),
      age = mean(boot_data$age),
      sex = "Female",
      diabetes = 0,
      hypertension = 0
    )
    
    # Predict protection probability
    pred_prob <- predict(model, newdata = pred_data, type = "response")
    pred_protection <- 1 - pred_prob
    
    # Find threshold
    idx <- which.min(abs(pred_protection - protection_level))
    return(antibody_range[idx])
  }, error = function(e) {
    return(NA)
  })
}

# Perform bootstrap (reduced number for demonstration)
cat("Performing bootstrap analysis (this may take a moment)...\n")
boot_results <- boot(data = cop_data, 
                    statistic = bootstrap_threshold, 
                    R = 100,  # Reduced for demonstration
                    protection_level = 0.8)

# Calculate confidence intervals
boot_ci <- boot.ci(boot_results, type = "perc")

cat("\n=== BOOTSTRAP RESULTS FOR 80% PROTECTION THRESHOLD ===\n")
cat("Original estimate:", round(boot_results$t0, 2), "\n")
cat("Bootstrap mean:", round(mean(boot_results$t, na.rm = TRUE), 2), "\n")
cat("95% CI:", round(boot_ci$percent[4], 2), "-", round(boot_ci$percent[5], 2), "\n")

# Plot bootstrap distribution
boot_plot <- ggplot(data.frame(threshold = boot_results$t), aes(x = threshold)) +
  geom_histogram(bins = 20, alpha = 0.7, fill = "steelblue") +
  geom_vline(xintercept = boot_results$t0, color = "red", linetype = "dashed") +
  labs(
    title = "Bootstrap Distribution of 80% Protection Threshold",
    x = "Antibody Threshold",
    y = "Frequency",
    caption = "Red line: original estimate"
  ) +
  theme_minimal()

print(boot_plot)

## 7. Results Interpretation

### 7.1 Key Findings Summary

In [ ]:
# Create comprehensive results summary
cat("\n", "="*60, "\n")
cat("                 CORRELATES OF PROTECTION ANALYSIS\n")
cat("                        RESULTS SUMMARY\n")
cat("="*60, "\n")

# Study population summary
cat("\n1. STUDY POPULATION:\n")
cat("   - Total participants:", nrow(cop_data), "\n")
cat("   - Infections observed:", sum(cop_data$infected), "\n")
cat("   - Overall attack rate:", round(mean(cop_data$infected) * 100, 1), "%\n")
cat("   - Mean age:", round(mean(cop_data$age), 1), "years\n")
cat("   - Female proportion:", round(mean(cop_data$sex == "Female") * 100, 1), "%\n")

# Antibody analysis
cat("\n2. ANTIBODY ANALYSIS:\n")
cat("   - Median antibody level (infected):", 
    round(median(cop_data$antibody_level[cop_data$infected == 1]), 2), "\n")
cat("   - Median antibody level (protected):", 
    round(median(cop_data$antibody_level[cop_data$infected == 0]), 2), "\n")
cat("   - ROC AUC:", round(auc_value, 3), "\n")
cat("   - Optimal threshold (Youden's J):", round(optimal_threshold, 2), "\n")

# Regression results
cat("\n3. REGRESSION ANALYSIS:\n")
antibody_or <- exp(coef(model_multivariate)["log_antibody"])
cat("   - Odds ratio per log-unit antibody:", round(antibody_or, 3), "\n")
cat("   - Interpretation: Each log-unit increase in antibody\n")
cat("     reduces infection odds by", round((1 - antibody_or) * 100, 1), "%\n")

# Protection thresholds
cat("\n4. PROTECTION THRESHOLDS:\n")
for(i in 1:length(protection_levels)) {
  cat("   -", protection_levels[i] * 100, "% protection:", 
      round(antibody_thresholds[i], 2), "\n")
}

# Statistical significance
p_value <- summary(model_multivariate)$coefficients["log_antibody", "Pr(>|z|)"]
cat("\n5. STATISTICAL SIGNIFICANCE:\n")
cat("   - P-value for antibody association:", 
    ifelse(p_value < 0.001, "<0.001", round(p_value, 3)), "\n")
cat("   - Conclusion:", 
    ifelse(p_value < 0.05, "Significant correlate of protection", "Not significant"), "\n")

cat("\n" , "="*60, "\n")

### 7.2 Clinical Interpretation

Based on our analysis, we can draw several important conclusions:

#### 7.2.1 Evidence for Correlate of Protection

1. **Statistical Association**: The antibody level shows a strong statistical association with protection (p < 0.001)
2. **Dose-Response Relationship**: Clear dose-response relationship observed between antibody levels and protection
3. **Discriminatory Ability**: ROC AUC indicates good discriminatory ability of antibody levels

#### 7.2.2 Threshold Interpretation

- **50% Protection**: Represents the minimum level for population-level benefit
- **80% Protection**: Clinically meaningful threshold for individual protection
- **90% Protection**: High-confidence threshold for vulnerable populations

#### 7.2.3 Clinical Implications

1. **Vaccine Development**: These thresholds can guide vaccine development targets
2. **Booster Strategies**: Can inform timing of booster doses
3. **Population Immunity**: Helps assess population-level protection
4. **Regulatory Decisions**: Provides evidence for vaccine approval

#### 7.2.4 Study Limitations

1. **Observational Design**: Cannot establish causation
2. **Single Biomarker**: Only evaluated antibody levels
3. **Follow-up Duration**: Limited to 12 months
4. **Population Specific**: Results may not generalize to all populations

## 8. Best Practices and Recommendations

### 8.1 Statistical Best Practices for CoP Studies

#### 8.1.1 Study Design Considerations

1. **Sample Size Planning**:
   - Power calculations should account for expected attack rates
   - Consider stratification by important covariates
   - Plan for loss to follow-up and missing data

2. **Endpoint Selection**:
   - Primary endpoint should be clinically meaningful
   - Consider multiple endpoints (infection, disease, severity)
   - Pre-specify endpoints to avoid bias

3. **Biomarker Selection**:
   - Include multiple immunological markers
   - Consider functional assays (neutralization)
   - Account for assay variability

#### 8.1.2 Analysis Methodology

1. **Model Selection**:
   - Use appropriate statistical models for outcome type
   - Consider non-linear relationships
   - Account for confounding variables

2. **Threshold Determination**:
   - Use multiple approaches (ROC, quantile regression)
   - Provide confidence intervals
   - Consider clinical meaningfulness

3. **Validation**:
   - Cross-validation within study
   - External validation when possible
   - Bootstrap for uncertainty quantification

#### 8.1.3 Reporting Guidelines

1. **Transparency**:
   - Report all analyses performed
   - Provide code and data when possible
   - Document assumptions and limitations

2. **Completeness**:
   - Include confidence intervals
   - Report effect sizes, not just p-values
   - Describe clinical significance

### 8.2 Regulatory Considerations

#### 8.2.1 Prentice Criteria

For a biomarker to be considered a valid correlate of protection, it should satisfy:

1. **Criterion 1**: Treatment (vaccination) significantly affects the biomarker
2. **Criterion 2**: Treatment significantly affects the clinical outcome
3. **Criterion 3**: Biomarker significantly predicts clinical outcome
4. **Criterion 4**: Treatment effect on outcome is fully explained by biomarker

#### 8.2.2 Regulatory Acceptance

- **FDA Guidance**: Follow FDA guidance on biomarker qualification
- **EMA Guidelines**: Consider EMA requirements for correlates
- **WHO Recommendations**: Align with WHO guidelines for vaccine evaluation

### 8.3 Future Directions

#### 8.3.1 Methodological Advances

1. **Machine Learning**: Explore ML approaches for threshold determination
2. **Multi-marker Models**: Combine multiple biomarkers
3. **Longitudinal Modeling**: Account for temporal changes
4. **Causal Inference**: Methods to establish causation

#### 8.3.2 Study Enhancements

1. **Longer Follow-up**: Assess durability of protection
2. **Diverse Populations**: Include various demographic groups
3. **Variant Tracking**: Monitor against emerging variants
4. **Mechanistic Studies**: Understand biological mechanisms

### 8.4 R Code Best Practices

#### 8.4.1 Code Organization

```r
# 1. Load libraries at the beginning
# 2. Set seed for reproducibility
# 3. Define functions before use
# 4. Use meaningful variable names
# 5. Comment your code extensively
# 6. Save intermediate results
# 7. Create publication-ready figures
```

#### 8.4.2 Documentation

- Document all analysis steps
- Include session information
- Version control your code
- Create reproducible reports

#### 8.4.3 Quality Assurance

- Validate results with independent analysis
- Check for coding errors
- Peer review statistical code
- Test with simulated data

---

## Conclusion

This statistical analysis plan provides a comprehensive framework for analyzing correlates of protection studies. The simulated analysis demonstrates key methodological approaches and provides R code for implementation. The identified correlates and thresholds can inform vaccine development, regulatory decisions, and public health strategies.

**Key Takeaways:**
1. Antibody levels show strong association with protection
2. Multiple analytical approaches strengthen conclusions
3. Confidence intervals provide important uncertainty quantification
4. Clinical interpretation requires domain expertise
5. Regulatory guidelines should inform study design and analysis

This framework can be adapted for specific pathogens, populations, and study designs while maintaining rigorous statistical standards.